In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/pope/popular/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,category,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a snowboard in the image?\nAnswer th...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(4553), tensor(4554), tensor(4555), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there an umbrella in the image?\nAnswer th...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(6205), tensor(6206), tensor(6207), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a person in the image?\nAnswer the q...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(10937), tensor(10938), tensor(10939), ...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a person in the image?\nAnswer the q...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(12189), tensor(12190), tensor(12191), ...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a car in the image?\nAnswer the ques...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(9487), tensor(9488), tensor(9489), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None


In [5]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [6]:
embeddings=np.concatenate(embeddings,axis=0)

In [7]:
embeddings.shape

(13624, 33, 4096)

In [8]:
gt_label=df['label'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [9]:
gt_label.shape

(13624,)

In [10]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [11]:
is_correct=[]
for i,x in enumerate(resps):
    if x.lower()=='no' and gt_label[i]==0:
        is_correct.append(1)
    elif x.lower()=='yes' and gt_label[i]==1:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [12]:
is_correct.shape

(13624,)

In [13]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 13624 responses. 11850 answer correctly and 1774 answer wrongly


In [ ]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
y_train = is_correct[train_idxs]
y_test = is_correct[test_idxs]


print(f'trainset length:{len(y_train)}')
print(f'testset length:{len(y_test)}')
print(f'trainset postive: {sum(y_train==1)} negative: {sum(y_train==0)}')
print(
    f'testset postive: {sum(y_test==1)} negative: {sum(y_test==0)}')

trainset length:10899
testset length:2725
trainset postive: 9450 negative: 1449
testset postive: 2400 negative: 325


In [15]:
all_embeddings = embeddings
all_embeddings.shape

(13624, 33, 4096)

In [16]:
embedding_train = all_embeddings[train_idxs]
embedding_test = all_embeddings[test_idxs]

In [17]:
best_layer=20

Train Linear Prob to Get Better Results

In [18]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=y_test

In [19]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [20]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.9394495412844037 AUROC:0.9496269230769231
